In [32]:
# global var to keep track of test results we have
ERRORS = []
PASSES = []

### Write Permission to Home Directory Test

In [23]:
import subprocess
import os
def shelltest(CMD):
    global ERRORS
    try:
        result = subprocess.check_output(CMD, shell=True, stderr=subprocess.STDOUT)
        return 0, result.decode('utf-8')
    except subprocess.CalledProcessError as e:
        return e.returncode, e.output.decode('utf-8')
    except Exception as e:  # new add: handle exceptions 
        return -1, str(e)

# Test to check write permissions to home directory
TEST = "WRITE PERMISSION TO HOME DIRECTORY"
CMD = f"touch {os.path.expanduser('~')}/test_write_permissions.tmp && echo 'Write Permission: Yes' && rm {os.path.expanduser('~')}/test_write_permissions.tmp || echo 'Write Permission: No'"

# Execute Test
e, output = shelltest(CMD)
if e == 0:
    PASSES.append("Write Permission to Home Directory test")
else:
    ERRORS.append(output)

### Environmental Variables test

In [ ]:
import os

#Verify environment variables are correct

NB_UID = int(os.environ['NB_UID'])
NB_GID = int(os.environ['NB_GID'])
NB_GROUP = os.environ['NB_GROUP']

XDG_CACHE_HOME = f"/home/{NB_USER}/.cache/"

UID_LOWER_BOUND = 2000  
UID_UPPER_BOUND = 60000 

GID_LOWER_BOUND = 2000  
GID_UPPER_BOUND = 60000 

EXPECTED_NB_GROUP = 'root'

err = []

def check_environment_test():
    if not (UID_LOWER_BOUND <= NB_UID <= UID_UPPER_BOUND):
         err.append(f"NB_UID {NB_UID} is not within the acceptable range: {UID_LOWER_BOUND}-{UID_UPPER_BOUND}.")


    if not (GID_LOWER_BOUND <= NB_GID <= GID_UPPER_BOUND):
         err.append(f"NB_GID {NB_GID} is not within the acceptable range: {GID_LOWER_BOUND}-{GID_UPPER_BOUND}.")


    if  NB_GROUP != EXPECTED_NB_GROUP:
         err.append("NB_GROUP does not match " + "'" + EXPECTED_NB_GROUP + "'")


    if NB_USER != 'jovyan':
         err.append("NB_USER does not match 'jovyan'.")


    if XDG_CACHE_HOME != '/home/jovyan/.cache/':
         err.append("XDD_CACHE_HOME does not match expected path: /home/jovyan/.cache/")


    if len(err) != 0:
         err.append("Environmental Variables test")

    else:
        s = '; '.join(err)
        print("Environmental Variables test ERROR: " + s)
    



### ASLR Test

In [ ]:
# run gdb 100 times on date binary stopping at first instruction and grep ld. If ld is found at same address (following 
# command has a value of 1 then ASLR is properly disabled.
# We want to disable ASLR so that students can use GDB properly in their environments.
e, count = shelltest("echo $(for ((i=0;i<100; i++)); do gdb -ex starti -ex quit -q --batch /usr/bin/date 2>/dev/null | grep ld; done | tee out | uniq | wc -l)")
if count == 1: 
    PASSES.append("ASLR test")
else:
    e2, output2 = shelltest("echo $(cat /proc/sys/kernel/randomize_va_space)")
    ERRORS.append(f"ASLR test ERROR: ASLR status is enabled with status {output2[0]} (0 is disabled, 1 is partial and 2 is full)")

### Network Test

In [26]:
# Curl test to check internet connectivity
e, output = shelltest("curl google.com")
if e == 0:
    PASSES.append("Network test")
else:
    ERRORS.append("Network test ERROR: " + output)

### Pip-Conda Test

In [27]:
TEST = "PIP PACKAGE INSTALLATION"
CMD = "pip install --user pytest"
e, output = shelltest(CMD)

if e == 0:
    PASSES.append("Pip-Conda test")
else:
    ERRORS.append("Pip-Conda test ERROR: " + output)

### Git and SSH Test

In [28]:
e, output = shelltest("readlink -f ~/.gitconfig")

if e == 0:
    PASSES.append("Git config test")
else:
    ERRORS.append("Git config test ERROR" + output)

In [29]:
e, output = shelltest("readlink -f /etc/ssh/ssh_config")

if e == 0:
    PASSES.append("ssh config test")
else:
    ERRORS.append("ssh config test ERROR" + output)

### Conda Directory Test

In [30]:
def check_permissions(dir_path):
    """Check if a directory is readable and writable."""
    global ERRORS
    try:
        readable = os.access(dir_path, os.R_OK)
        writable = os.access(dir_path, os.W_OK)
        return readable, writable
    except Exception as e:
        ERRORS.append("Conda directory r/w test" + f"checking permissions for {dir_path}.")
        return False, False

# Identify the conda directories
conda_base_dir = os.path.abspath(os.path.join(os.path.dirname(os.sys.executable), ".."))
conda_env_dir = os.environ.get('CONDA_PREFIX', '')

unaccessible_dirs = 0

for dir_name, dir_path in [('Conda Base Directory', conda_base_dir), ('Conda Environment Directory', conda_env_dir)]:
    readable, writable = check_permissions(dir_path)

    if not (readable and writable):
        unaccessible_dirs += 1
if unaccessible_dirs == 0:
    PASSES.append("Conda directory r/w test")

In [31]:
if len(ERRORS) > 0:
    print("FAILED TESTS:")
    print('-' + '\n-'.join(ERRORS))
    print('\n')
    print("PASSED TESTS:")
    print('-' + '\n-'.join(PASSES))
else:
    print("ALL TESTS PASS")
    print('-' + '\n-'.join(PASSES))

FAILED TESTS:
-ASLR test ERROR: ASLR status is enabled with status 2 (0 is disabled, 1 is partial and 2 is full)
-Network test ERROR: /bin/sh: 1: curl: not found



PASSED TESTS:
-Write Permission to Home Directory test
-Pip-Conda test
-Git config test
-ssh config test
